# This is the code to test Neo 6M GPS Module

Check if the device identifies the GPS module  
It will be named ACM 0 (if connected by UBS). If not unplug it and replug to identify which port is it assigned to.

In [ ]:
!ls /dev/tty*

### Now testing
(Potential errors would be: "ValueError: could not convert string to float:"  
Then wait untill GPS locks to a satilite)

In [ ]:
import serial
import time
import string
# GPS (UART)

ser = serial.Serial("/dev/ttyACM0") # Address of GPS Device connected via USB
gpgga_info = "$GPGGA," # Extract GPGGA line from GPS
GPGGA_buffer = 0
NMEA_buff = 0


def convert_to_degrees(raw_value):
    decimal_value = raw_value/100.00
    degrees = int(decimal_value)
    mm_mmmm = (decimal_value - int(decimal_value))/0.6
    position = degrees + mm_mmmm
#     position = "%.4f" %(position)
    return position


try:
    while True:
       
        received_data = (str)(ser.readline()) #read NMEA string received
        GPGGA_data_available = received_data.find(gpgga_info)   #check for NMEA GPGGA string
        
        if (GPGGA_data_available>0): # when GPS is active
            GPGGA_buffer = received_data.split("$GPGGA,",1)[1]  #store data coming after “$GPGGA,” string
            NMEA_buff = (GPGGA_buffer.split(','))
            nmea_time = []
            nmea_latitude = []
            nmea_longitude = []
            nmea_time = float(NMEA_buff[0]) + 53000                    #extract time from GPGGA string in Indian Standard Time
            nmea_latitude = NMEA_buff[1]                #extract latitude from GPGGA string
            nmea_latitude_dir = NMEA_buff[2]                #Lat direction
            nmea_longitude = NMEA_buff[3]               #extract longitude from GPGGA string
            nmea_long_dir = NMEA_buff[4]                #Long direction  

            GPS_Fix = NMEA_buff[5]
            satellite_count = NMEA_buff[6]
            Alt = NMEA_buff[8]

            lat = float(nmea_latitude)
            lat = convert_to_degrees(lat)
            longi = float(nmea_longitude)
            longi = convert_to_degrees(longi)
            
            print("Time:",nmea_time)
            print("Latitude:", lat)
            print("Longitude:",longi)
            print("Satilite Count:",satellite_count,"Altitude (m):",Alt)
            time.sleep(1)
            
except KeyboardInterrupt:
    sys.exit(0)